In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from database.market import Market
from datetime import datetime, timedelta
from tqdm import tqdm
from extractor.tiingo_extractor import TiingoExtractor

In [2]:
market = Market()
market.connect()
dates = market.retrieve_date_range()
sp5 = market.retrieve("sp500")
market.disconnect()

In [3]:
stuff = pd.to_datetime(dates["date"])

In [4]:
start = (stuff.max() + timedelta(days=1)).strftime("%Y-%m-%d")
end = datetime.now().strftime("%Y-%m-%d")
print(start,end)

2021-11-16 2021-11-17


In [5]:
market.connect()
for ticker in tqdm(sp5["Symbol"].unique()):
    try:
        if "." in ticker:
            ticker = ticker.replace(".","-")
        data = TiingoExtractor.extract(ticker,start,end)
        data["ticker"] = ticker
        market.store("prices",data)
    except Exception as e:
        print(str(e))
market.disconnect()

 16%|████████████████████▊                                                                                                             | 81/505 [00:52<04:41,  1.51it/s]

Expecting value: line 1 column 1 (char 0)
'NoneType' object does not support item assignment


 61%|██████████████████████████████████████████████████████████████████████████████▏                                                  | 306/505 [03:19<02:23,  1.39it/s]

project_market prices documents must be a non-empty list


 99%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏| 502/505 [05:28<00:02,  1.25it/s]

...

Expecting value: line 1 column 1 (char 0)
'NoneType' object does not support item assignment


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋| 504/505 [05:30<00:00,  1.40it/s]

Expecting value: line 1 column 1 (char 0)
'NoneType' object does not support item assignment


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 505/505 [05:30<00:00,  1.53it/s]

Expecting value: line 1 column 1 (char 0)
'NoneType' object does not support item assignment


In [6]:
data

In [7]:
analysis = []
market.connect()
for ticker in tqdm(sp5["Symbol"].unique()):
    report = {}
    try:
        test = market.retrieve_ticker_data(ticker)
        test["date"] = pd.to_datetime(test["date"])
        test["year"] = [x.year for x in test["date"]]
        test = test[test["year"]==test["year"].max()].sort_values("date")      
        test["quarter"] = [x.quarter for x in test["date"]]
        test["week"] = [x.week for x in test["date"]]
        test["month"] = [x.month for x in test["date"]]
        periods = ["year","quarter","month","week"]
        ep = test.tail(1)["adjClose"].item()
        # periods = ["year"]
        for period in periods:
            current = test.tail(1)[period].item()
            window = test[(test[period]==current)].sort_values("date")
            sp = window.iloc[0]["adjClose"].item()
            gain = (ep - sp) / sp
            report[f"{period}_gain"] = gain
        report["day_gain"] = (ep - test.tail(2).iloc[0]["adjClose"].item())/test.tail(2).iloc[0]["adjClose"].item()
        report["ticker"] = ticker
        analysis.append(report)
    except Exception as e:
        print(str(e))
market.disconnect()

 13%|████████████████▋                                                                                                                 | 65/505 [02:00<13:28,  1.84s/it]

'date'


 16%|████████████████████▎                                                                                                             | 79/505 [02:26<13:04,  1.84s/it]

'date'


 99%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏| 502/505 [15:38<00:05,  1.86s/it]

...

'date'


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋| 504/505 [15:42<00:01,  1.84s/it]

'date'


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 505/505 [15:44<00:00,  1.87s/it]

'date'


In [8]:
a = pd.DataFrame(analysis).merge(sp5.rename(columns={"Symbol":"ticker"})[["ticker","Security","GICS Sector"]],on="ticker",how="left")
a.to_csv("analysis.csv")

In [9]:
period = "day"
a.sort_values(f"{period}_gain",ascending=False).head(50)

,year_gain,quarter_gain,month_gain,week_gain,day_gain,ticker,Security,GICS Sector
148,0.239854,0.353235,0.197093,0.000000,0.172533,DLTR,Dollar Tree,Consumer Discretionary
389,0.242348,0.412555,0.348639,0.000000,0.102280,QCOM,Qualcomm,Information Technology
455,0.531412,0.278018,0.208826,0.000000,0.061950,UAA,Under Armour (Class A),Consumer Discretionary
486,0.550997,0.053606,0.036298,0.000000,0.054913,WST,West Pharmaceutical Services,Health Care
456,0.490040,0.242525,0.179191,0.000000,0.054016,UA,Under Armour (Class C),Consumer Discretionary
174,0.666376,0.403092,0.177038,0.000000,0.053647,ETSY,Etsy,Consumer Discretionary
229,0.511307,0.189383,0.070536,0.000000,0.052867,HD,Home Depot,Consumer Discretionary
255,0.514949,0.107597,0.033412,0.000000,0.050495,IQV,IQVIA,Health Care
240,0.457761,0.117283,0.036993,0.000000,0.049383,INFO,IHS Markit,Industrials
399,0.262805,0.016323,0.005775,0.000000,0.045799,RMD,ResMed,Health Care


In [10]:
a.groupby("GICS Sector").mean().sort_values("day_gain",ascending=False).head(20)

,year_gain,quarter_gain,month_gain,week_gain,day_gain
GICS Sector,,,,,
Consumer Discretionary,0.336451,0.099661,0.035486,0.000147,0.010036
Utilities,0.112882,0.039131,-0.003286,-0.001376,0.006949
Information Technology,0.292022,0.094681,0.045002,0.002460,0.006265
Industrials,0.307670,0.084365,0.026109,-0.000758,0.003217
Financials,0.415815,0.058696,0.004400,-0.000391,0.001880
Energy,0.665946,0.073890,-0.016737,0.001307,0.001657
Consumer Staples,0.092853,0.038229,0.024927,-0.000265,0.001566
Real Estate,0.458597,0.071126,0.015847,-0.000661,0.000666
Health Care,0.192971,0.014141,-0.007862,0.000377,-0.000677
